In [102]:
import pandas as pd
import sklearn.model_selection as model_selection

from keras.layers import Dense, LSTM
from keras.models import Sequential
from sklearn.preprocessing import MinMaxScaler

In [103]:
NUM_COLS = ['Zero', 'One', 'Two', 'Three', 'Four', 'Five', 'Six', 'Seven', 'Eight', 'Nine']

In [104]:
# Đọc dữ liệu từ tập tin CSV
df = pd.read_csv('lottery.csv')

In [105]:
# Chuẩn bị dữ liệu
X = df.drop(columns=['Date']).values
y = df[NUM_COLS].values

In [106]:
# Chia dữ liệu thành tập huấn luyện và tập kiểm tra
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=0.2, random_state=42)

In [107]:
# Chuẩn hóa dữ liệu
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [108]:
# Reshape lại dữ liệu để phù hợp với input của mạng LSTM
X_train_reshaped = X_train_scaled.reshape((X_train_scaled.shape[0], 1, X_train_scaled.shape[1]))
X_test_reshaped = X_test_scaled.reshape((X_test_scaled.shape[0], 1, X_test_scaled.shape[1]))

In [109]:
# Xây dựng mô hình LSTM
model = Sequential()
model.add(LSTM(50, input_shape=(X_train_reshaped.shape[1], X_train_reshaped.shape[2])))
model.add(Dense(10))  # 10 đầu ra tương ứng với số lần xuất hiện từ zero đến nine
model.compile(loss='mse', optimizer='adam')

In [110]:
# Huấn luyện mô hình
history = model.fit(X_train_reshaped, y_train, epochs=100, batch_size=16, validation_data=(X_test_reshaped, y_test), verbose=2)

Epoch 1/100
350/350 - 2s - loss: 50.7983 - val_loss: 9.7125 - 2s/epoch - 5ms/step
Epoch 2/100
350/350 - 0s - loss: 9.5440 - val_loss: 9.5814 - 385ms/epoch - 1ms/step
Epoch 3/100
350/350 - 0s - loss: 9.5198 - val_loss: 9.5656 - 381ms/epoch - 1ms/step
Epoch 4/100
350/350 - 0s - loss: 9.4958 - val_loss: 9.5366 - 374ms/epoch - 1ms/step
Epoch 5/100
350/350 - 0s - loss: 9.4671 - val_loss: 9.5069 - 380ms/epoch - 1ms/step
Epoch 6/100
350/350 - 0s - loss: 9.4274 - val_loss: 9.4608 - 397ms/epoch - 1ms/step
Epoch 7/100
350/350 - 0s - loss: 9.3660 - val_loss: 9.3910 - 392ms/epoch - 1ms/step
Epoch 8/100
350/350 - 0s - loss: 9.2703 - val_loss: 9.2716 - 383ms/epoch - 1ms/step
Epoch 9/100
350/350 - 0s - loss: 9.0219 - val_loss: 8.8613 - 379ms/epoch - 1ms/step
Epoch 10/100
350/350 - 0s - loss: 8.4171 - val_loss: 8.0338 - 413ms/epoch - 1ms/step
Epoch 11/100
350/350 - 0s - loss: 7.3262 - val_loss: 6.7021 - 444ms/epoch - 1ms/step
Epoch 12/100
350/350 - 0s - loss: 5.7819 - val_loss: 4.9826 - 389ms/epoch - 

In [111]:
# Đánh giá mô hình trên tập kiểm tra
loss = model.evaluate(X_test_reshaped, y_test, verbose=0)
print(f'Test Loss: {loss}')

Test Loss: 0.010438773781061172


In [112]:
# Lưu mô hình
model.save('lottery.keras')

In [113]:
# Dự đoán số lần xuất hiện của các cột từ zero đến nine cho hàng cuối cùng trong dữ liệu
last_row_scaled = scaler.transform(X[-1:])
predictions = model.predict(last_row_scaled.reshape((last_row_scaled.shape[0], 1, last_row_scaled.shape[1])))
print(f'Predictions: {predictions}')

1/1 [==============================] - 0s 237ms/step
Predictions: [[ 9.984458  11.979424  12.005852  12.028193  10.037702  16.04603
  11.947015   9.998369   5.9876814  6.9534693]]


In [114]:
# Tạo dictionary giữa các cột từ zero đến nine và dự đoán tương ứng
predictions_dict = {col_name: prediction for col_name, prediction in zip(NUM_COLS, predictions[0])}
print(f'Dictionary giữa dự đoán và các cột tương ứng: {predictions_dict}')

Dictionary giữa dự đoán và các cột tương ứng: {'Zero': 9.984458, 'One': 11.979424, 'Two': 12.005852, 'Three': 12.028193, 'Four': 10.037702, 'Five': 16.04603, 'Six': 11.947015, 'Seven': 9.998369, 'Eight': 5.9876814, 'Nine': 6.9534693}


In [115]:
# Kết quả dự đoán cao nhất và thấp nhất
predictions_items = predictions_dict.items()
print(f'''Cặp số có giá trị dự đoán cao nhất: {dict(sorted(predictions_items, key=lambda x: x[1], reverse=True)[:2])}
Cặp số có giá trị dự đoán thấp nhất: {dict(sorted(predictions_items, key=lambda x: x[1])[:2])}''')

Cặp số có giá trị dự đoán cao nhất: {'Five': 16.04603, 'Three': 12.028193}
Cặp số có giá trị dự đoán thấp nhất: {'Eight': 5.9876814, 'Nine': 6.9534693}
